# Solow model augmented with human capital

In this project we augment the standard Solow model with human capital and see how this affects the steady state level technology adjusted capital labour ratio  and output.

Imports and set magics:

In [1]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

Consider the **augmented Solow-model with human capital** where:

1. $K_t$ is capital
2. $H_t$ is total human capital ($H_t = h_t * L_t$)
3. $L_t$ is labor (growing with a constant rate of $n$)
4. $A_t$ is technology (growing with a constant rate of $g$)
5. $Y_t = F(K_t,A_tL_t)$ is GDP
6. $s_K$ is the saving rate of capital
7. $s_H$ is the saving rate of human capital

**Saving** is a constant fraction of GDP

$$ 
S_t = (s_{K} + s_{H})Y_t,\,s_{K},s_{H}\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=s_{K}Y_{t}+(1-\delta)K_{t}=s_{K}F(K_{t}H_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

and that **human capital accumulates** according to

$$
H_{t+1}=s_{H}Y_{t}+(1-\delta)H_{t}=s_{H}F(K_{t}H_{t},A_{t}L_{t})+(1-\delta)H_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t}H_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},\tilde{h}_{t},1)\equiv f(\tilde{k}_{t},\tilde{h}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio and $\tilde{h}_t = \frac{H_t}{A_{t}L_{t}}$ is the technology adjusted human capital-labor ratio

The **transition equations** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[s_{K}f(\tilde{k}_{t},\tilde{h}_{t})+(1-\delta)\tilde{k}_{t}]
$$

$$
\tilde{h}_{t+1}= \frac{1}{(1+n)(1+g)}[s_{H}f(\tilde{k}_{t},\tilde{h}_{t})+(1-\delta)\tilde{h}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t}H_{t},A_{t}L_{t})=K_{t}^{\alpha}H_{t}^{\phi}(A_{t}L_{t})^{1-\alpha-\phi}\Rightarrow f(\tilde{k}_{t},\tilde{h}_{t})=\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\phi}
$$



## Analytical solution

To find the steady states of the two production factors we set $k_t = k_{t+1}$ and $h_t=h_{t+1}$

$$
\tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[s_{K}f(\tilde{k}^{\ast},\tilde{h}^{\ast})+(1-\delta)\tilde{k}^{\ast}]
$$

$$
\tilde{h}^{\ast}= \frac{1}{(1+n)(1+g)}[s_{H}f(\tilde{k}^{\ast},\tilde{h}^{\ast})+(1-\delta)\tilde{h}^{\ast}]
$$

In [2]:
k = sm.symbols('k')
h = sm.symbols('h')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
phi = sm.symbols('phi')
sk = sm.symbols('s_K')
sh = sm.symbols('s_H')
g = sm.symbols('g')
n = sm.symbols('n')

In [3]:
# Define f(k)
f = k**alpha*h**phi

# Transition equation for k and h to find SS
ssk = sm.Eq(0,(sk*f-(n+g+delta+n*g)*k)/((1+n)*(1+g)))
ssh = sm.Eq(0,(sh*f-(n+g+delta+n*g)*h)/((1+n)*(1+g)))

# Isolate for k in transition eq.
k1 = sm.solve(ssk,k)[0]

# Display solution
k1

((delta + g*n + g + n)/(h**phi*s_K))**(1/(alpha - 1))

In [4]:
ssh_sub = ssh.subs(k,k1)
ssh_sub

Eq(0, (-h*(delta + g*n + g + n) + h**phi*s_H*(((delta + g*n + g + n)/(h**phi*s_K))**(1/(alpha - 1)))**alpha)/((g + 1)*(n + 1)))

In [5]:
#solve for h in ssh_sub
#h_ss = sm.solve(ssh_sub,h)
#h_ss
#we can not get a solution for h using the sympy solver, do not know why?

In [6]:
#The steady state expresion for k and h: 
k_ss = sm.Eq(k,(((sk**(1-phi)*sh**phi)/(n+g+delta+n*g))**(1/(1-alpha-phi))))
h_ss = sm.Eq(h,(((sk**alpha*sh**(1-alpha))/(n+g+delta+n*g))**(1/(1-alpha-phi))))
k_ss

Eq(k, (s_H**phi*s_K**(1 - phi)/(delta + g*n + g + n))**(1/(-alpha - phi + 1)))

In [7]:
#null clines: we isolate both transition equations for h
h_k = sm.solve(ssk,h)[0]
h_h = sm.solve(ssh,h)[0]
h_k

(k**(1 - alpha)*(delta + g*n + g + n)/s_K)**(1/phi)

In [9]:
# Use sm.lambdify to turn the solution into 
k_ss_func = sm.lambdify(args=(sk,sh,g,n,delta,alpha, phi),expr=k_ss.rhs)
h_ss_func = sm.lambdify(args=(sk,sh,g,n,delta,alpha, phi),expr=h_ss.rhs)

# Evaluate the function
k_an = k_ss_func(0.2, 0.2, 0.02,0.01,0.1,1/3, 1/3)
h_an = h_ss_func(0.2, 0.2, 0.02,0.01,0.1,1/3, 1/3)
print(k_an, h_an)

3.6245745297718917 3.6245745297718917


## Numerical solution

We now solve the model numerically. We start of with defining the parameters:

In [10]:
sk_num = 0.2
sh_num = 0.2
g_num = 0.02
n_num = 0.01
alpha_num = 1/3
phi_num = 1/3
delta_num = 0.1

We firstly solve the standard solow model without human capital

In [21]:
# Define f
f_stand = lambda k_stand: k_stand**alpha_num

# Define the objective function
obj_kss = lambda kss: kss - (sk_num*f_stand(kss) + (1-delta_num)*kss)/((1+g_num)*(1+n_num))
result = optimize.root_scalar(obj_kss,bracket=[0.1,100],method='brentq')

print('the steady state for k is in the standard model',result.root) 

the steady state for k is in the standard model 1.9038315392313185


We use the found steady values of ${k}^{*}$ to compute the steady state value of $y^{*}$

In [12]:
y_stand_star = f_stand(result.root)
y_stand_star

1.2393943320395886

We then solve the augmented solow model with human capital

In [13]:
# Define f
f = lambda k_num, h_num: k_num**alpha_num*h_num**phi_num

# Define the objective function
def equations(vars):
    k_num, h_num = vars
    eq1 = (sk_num*f(k_num, h_num)-(n_num+g_num+delta_num+n_num*g_num)*k_num)/((1+n_num)*(1+g_num))
    eq2 = (sh_num*f(k_num, h_num)-(n_num+g_num+delta_num+n_num*g_num)*h_num)/((1+n_num)*(1+g_num))
    return [eq1, eq2]

# Find result
k_list = []
h_list = []
for i, j in zip(np.linspace(0.1, 10, 50), np.linspace(0.1, 10, 50)):
    k_num, h_num = optimize.fsolve(equations, (i, j))
    k_list.append(k_num)
    h_list.append(h_num)


# Print result
print(k_list)
print(h_list)

#as seen low values of initial guesses does not converge to the analytical solution to the steady state.

[0.1, 0.3020408163265306, 0.5040816326530613, 0.7061224489795919, 0.9081632653061225, 3.624574529771892, 3.6245745297718894, 3.6245745297718948, 3.624574529771866, 3.6245745297718943, 3.6245745297718277, 3.624574529771893, 3.624574529771893, 3.6245745297718948, 3.6245745297718654, 3.6245745297718943, 3.624574529771895, 3.624574529771894, 3.624574529771896, 3.6245745297718948, 3.6245745297718925, 3.624574529771894, 3.6245745297719068, 3.624574529772002, 3.624574529771892, 3.624574529771892, 3.624574529771895, 3.6245745297718956, 3.624574529771893, 3.6245745297718934, 3.624574529771893, 3.6245745297718948, 3.624574529771895, 3.6245745297718974, 3.6245745297718988, 3.6245745297719028, 3.6245745297719103, 3.6245745297719263, 3.6245745297719565, 3.6245745297720067, 3.624574529771892, 3.6245745297718943, 3.624574529771894, 3.624574529771893, 3.6245745297718956, 3.6245745297718925, 3.6245745297718934, 3.624574529771893, 3.6245745297718934, 3.6245745297718956]
[0.1, 0.3020408163265306, 0.50408

C:\Users\PCUser\AppData\Local\Temp\ipykernel_5908\3487670250.py:2: RuntimeWarning: invalid value encountered in scalar power
  f = lambda k_num, h_num: k_num**alpha_num*h_num**phi_num
c:\Users\PCUser\anaconda3\Lib\site-packages\scipy\optimize\_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


We use the found steady values of $\tilde{k}_{t}^{*}$ and $\tilde{h}_{t}^{*}$ to compute the steady state value of $y_t$

In [14]:
y_star = f(k_an, h_an)
y_star

2.359598018881502

In [17]:
h_k_nc = sm.lambdify(args=(sk,sh,g,n,delta,alpha, phi, k),expr=h_k)
h_h_nc = sm.lambdify(args=(sk,sh,g,n,delta,alpha, phi, k),expr=h_h)

# Evaluate the function
h_k_nc_list = []
h_h_nc_list = []

k_val_list = np.linspace(0,2,1000)

for k_val in k_val_list:
    h_k_num = h_k_nc(0.2, 0.2, 0.02,0.01,0.1,1/3, 1/3, k_val)
    h_h_num = h_h_nc(0.2, 0.2, 0.02,0.01,0.1,1/3, 1/3, k_val)
    h_k_nc_list.append(h_k_num)
    h_h_nc_list.append(h_h_num)


<lambdifygenerated-8>:2: RuntimeWarning: divide by zero encountered in scalar power
  return (k**(-alpha)*(delta + g*n + g + n)/s_H)**((phi - 1)**(-1.0))


# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

In [24]:
# Define a function to plot the data
def plot_data(sk, sh, g, n, delta, alpha, phi):
    h_k_nc_list = []
    h_h_nc_list = []
    k_val_list = np.linspace(0,4,1000)
    for k_val in k_val_list:
        h_k_num = h_k_nc(sk, sh, g, n, delta, alpha, phi, k_val)
        h_h_num = h_h_nc(sk, sh, g, n, delta, alpha, phi, k_val)
        h_k_nc_list.append(h_k_num)
        h_h_nc_list.append(h_h_num)
    plt.plot(k_val_list, h_k_nc_list, label='$\Delta k = 0$')
    plt.plot(k_val_list, h_h_nc_list, label='$\Delta h = 0$')
    plt.xlabel('$k_t$ values')
    plt.ylabel('$h_t$ values')
    plt.title('Plot of h_k and h_h')
    plt.legend()
    plt.show()

# Create sliders for the parameters
sk_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.2, description='sk:')
sh_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.2, description='sh:')
g_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.02, description='g:')
n_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.01, description='n:')
delta_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.1, description='delta:')
alpha_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=1/3, description='alpha:')
phi_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=1/3, description='phi:')

# Create an interactive plot
widgets.interactive(plot_data, sk=sk_slider, sh=sh_slider, g=g_slider, n=n_slider, delta=delta_slider, alpha=alpha_slider, phi=phi_slider)

interactive(children=(FloatSlider(value=0.2, description='sk:', max=1.0, step=0.01), FloatSlider(value=0.2, de…

The plot illustrates that the implications of the model are highly dependes on the model parameter. For example considering savings rate in physical capital, we see that a higher savings rate leads to a higher technology adjusted physical capital labour ratio ceteris paribus. This is due to increased allocation of output into physical capital. Due to the increase in output from the higher level of physical capital this also implies a increase in the level of human capital. 

# Conclusion

In this project we have augmented the standard Solow model with labor, capital and technology with human capital. We see that the inclusion of this increases the steady state technology adjusted capital labour ratio and output. This is due to the complentarity effects between human and physical capital. These effects improves the marginal product of physical and human capitl respectively. However this conclusion depends on the chosen parameters. 